<a href="https://colab.research.google.com/github/ErikSeguinte/movie_data/blob/master/processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import cufflinks as cf
import numpy as np
from plotly import graph_objs as go
import altair as alt

In [0]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [0]:
cf.set_config_file(offline=True)

In [0]:
merge_keys = {'left_index':True, 'right_index':True}

* I previously pulled CSV files from Kaggle, but the files were too big to host on github.
* I imported the files I wanted into pandas, and then exported them back out as compressed pickles.
* I was able to compress a 700MB csv to a 3 MB Pickle

In [18]:
try: 
    movies = pd.read_pickle('data/movies.pkl.xz')
    #ratings = pd.read_pickle('data/ratings2.pkl.xz')
    #cpi = pd.read_csv('data/cpi.csv')
    
except:
    # Download pickles from github
    #!wget https://github.com/ErikSeguinte/movie_data/raw/master/data/ratings2.pkl.xz
    !wget https://github.com/ErikSeguinte/movie_data/raw/master/data/movies.pkl.xz
    #!wget 'https://datahub.io/core/cpi/r/cpi.csv'
    #cpi = pd.read_csv('data/cpi.csv')
    movies = pd.read_pickle('movies.pkl.xz')
    #ratings = pd.read_pickle('ratings2.pkl.xz')

--2020-03-02 20:43:52--  https://github.com/ErikSeguinte/movie_data/raw/master/data/movies.pkl.xz
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ErikSeguinte/movie_data/master/data/movies.pkl.xz [following]
--2020-03-02 20:43:52--  https://raw.githubusercontent.com/ErikSeguinte/movie_data/master/data/movies.pkl.xz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7917036 (7.5M) [application/octet-stream]
Saving to: ‘movies.pkl.xz.1’

movies.pkl.xz.1     100%[===================>]   7.55M  --.-KB/s    in 0.1s    

2020-03-02 20:43:53 (65.0 MB/s) - ‘movies.pkl.xz.1’ saved [7917036/7917036]



## Clean Movie DF

In [42]:
movies.dtypes

adult                            object
belongs_to_collection            object
budget                          float64
genres                           object
homepage                         object
imdb_id                          object
original_language                object
original_title                   object
overview                         object
popularity                       object
poster_path                      object
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
status                           object
tagline                          object
title                            object
video                            object
vote_average                    float64
vote_count                      float64
year                            float64
dtype: object

* Movies Dataframe has malformed data. `id` Should be numeric.
* After inspection, it looks like there are rows that are missing a comma somewhere, making columns not line up, and adding the wrong data to columns. Let's clean those up.
* All malformed rows have strings for IDs instead of numeric, so we will coerce them into numeric columns, and strings will be returned as `NaN`, which we'll then drop.

* `budget` and `revanue` should also be numeric, but Nans won't be dropped






In [0]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
movies = movies[movies['id'].notnull()]
movies = movies.set_index('id')

In [0]:
def to_numeric(df, labels):
    
    for label in labels:
        df[label] = pd.to_numeric(movies[label], errors='coerce').copy()
    return df

In [0]:
movies = to_numeric(movies, ['budget', 'revenue', 'vote_average'])

In [0]:
movies['release_date'] =pd.to_datetime(movies['release_date'], infer_datetime_format= True)

In [0]:
movies['year'] = movies['release_date'].dt.year

In [0]:
clean_movies = movies[['title', 'release_date','budget', 'revenue','year' ,'runtime', 'vote_average', 'vote_count']]

## Process User Reviews
* User reviews come in a collection of individual reviews where a review gives a movie a score of 1 to 5.
* We will take the mean ratings for each movie

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
scaler = StandardScaler()
mean_rating = pd.DataFrame(scaler.fit_transform(ratings.groupby('movieId')[['rating']].mean()), columns = ["rating"])
#mean_rating = pd.DataFrame(ratings.groupby('movieId')[['rating']].mean(), columns = ["rating"])

In [0]:
# Aggregate mean ratings and number of votes per movie
movie_ratings =pd.DataFrame(ratings.groupby('movieId')[['rating']].agg(['mean', 'count']))['rating']
movie_ratings = movie_ratings.rename({'mean': 'rating', 'count': 'num_votes'}, axis = 1)


In [0]:
ratings = None

* Lets drop any movies with less than 10 votes. Those are more easily swayed by outliers and aren't reliable.

In [0]:
movie_ratings = movie_ratings[~(movie_ratings['num_votes'] < 100)]

* And now we merge the averaged ratings back with the movie database.
* Note that not all movies are present in the user votings.

In [0]:
movie_ratings = clean_movies.merge(movie_ratings, left_index = True, right_index=True)

In [0]:
movie_ratings[['title', 'rating']].nlargest(10, 'rating')

* The movie Database also provides a rating and suffer from a similar problem of some movies having a tiny sample size.

In [0]:
movie_ratings[['title', 'vote_average', 'vote_count']].sort_values(by='vote_average', ascending = False).nlargest(10, 'vote_average')

In [0]:
movie_ratings[['title', 'revenue']].nlargest(10, 'revenue')

## Inflation
* Inflation means that a 1940 dollar is worth more than a 2020 dollar. Let's adjust Revenue for that.
* The Consumer price index can be used to convert to standarized dollars.
* Here, we'll be using 2014 dollars.
* Years later than 2014 will not be adjusted.
$$ \textrm{adjusted dollars} = \frac{\textrm{New CPI}}{\textrm{Base CPI}}$$
* where x is the current cpi and y is the cpi of that year 

In [0]:
!wget 'https://datahub.io/core/cpi/r/cpi.csv'

In [0]:
cpi = cpi[cpi['Country Name'] == 'United States'][['Year', 'CPI']]

In [0]:
cpi = cpi.set_index(cpi['Year'])

In [0]:
def adjust_dollars(value, year):
    year = int(year)
    try:
        current = cpi.loc[2014,'CPI']
        base = cpi.loc[year,'CPI']
        adjusted_value = value * (current/base)
        return adjusted_value
    except: 
        return value

In [0]:
clean_movies['year']= clean_movies['release_date'].dt.year

In [0]:
df = clean_movies[clean_movies['revenue'].notnull() & clean_movies['year'].notnull()]
df

In [0]:
adjusted = pd.DataFrame([adjust_dollars(x,y) for x,y in zip(df['revenue'], df['year'])], index = df.index, columns = ['adjusted_revenue'])

In [0]:
clean_movies.merge(adjusted, left_index=True, right_index=True).nlargest(10,'adjusted_revenue')

In [0]:
movie_ratings.corr()

In [0]:
# Adding a year and decade to examine trends over time
movie_ratings['year'] = movie_ratings['release_date'].dt.year

In [0]:
movie_ratings['decade'] = [x - (x%10) for x in movie_ratings['year']]

In [0]:
#enable_plotly_in_cell()
movie_ratings.groupby('year')['vote_average'].mean().iplot(kind='bar')

In [0]:
alt.Chart(movie_ratings).mark_bar().encode(
    alt.Y('mean(vote_average)'),
    alt.X('year')
)

In [0]:
alt.Chart(movie_ratings).mark_bar().encode(
    alt.Y('mean(rating)'),
    alt.X('year')
)

In [0]:
alt.Chart(movie_ratings).mark_bar().encode(
    alt.Y('mean(vote_average)'),
    alt.X('decade')
)

In [0]:
movie_ratings

In [0]:
# enable_plotly_in_cell()
movie_ratings.groupby('decade')['rating'].mean().iplot(kind='bar')

* I'd like to compare the votes from TMB to the user ratings, but they are on different scales. We'll use standard scaler to normalize them so we can more easily compare.

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
to_scale = movie_ratings[movie_ratings['vote_average'].notnull() & movie_ratings['year'].notnull() & movie_ratings['rating'].notnull()][['vote_average', 'rating']]
scaled = scaler.fit_transform(to_scale)
movie_ratings = movie_ratings.merge(
    pd.DataFrame(
        scaled,
        index = to_scale.index,
        columns = ['scaled_tmdb_vote', 'scaled_user_rating']
    ),
    left_index = True,
    right_index = True,
)

In [0]:
movie_ratings

In [0]:
traces = [
    go.Bar(name='TMDB rating',
        x = movie_ratings.groupby('year')['scaled_tmdb_vote'].mean().index,
        y = movie_ratings.groupby('year')['scaled_tmdb_vote'].mean()
    ),
        go.Bar(name='user rating',
        x = movie_ratings.groupby('year')['scaled_user_rating'].mean().index,
        y = movie_ratings.groupby('year')['scaled_user_rating'].mean()
    )
]

go.Figure(data = traces,
    layout_xaxis_tick0 = 1890
)

In [0]:
from sklearn.decomposition import PCA

pca = PCA(1)

pca_df = pd.DataFrame(pca.fit_transform(scaled), index=to_scale.index, columns = ['PCA'])

movie_ratings = movie_ratings.merge(pca_df, left_index = True, right_index = True)
movie_ratings.head(2)

In [0]:
movie_ratings[['title', 'PCA']].nlargest(10, 'PCA')

In [0]:
# enable_plotly_in_cell()
trace = go.Box(
    x = movie_ratings[movie_ratings['decade'].notnull()]['decade'],
    y = movie_ratings[movie_ratings['decade'].notnull()]['rating'],
    
)
go.Figure(
    trace,
    layout_xaxis_title = "Decade",
    layout_yaxis_title = "Movie Rating",
    layout_title = "Movie Ratings by decade"
)


In [0]:
movie_ratings['q_budget'] = pd.qcut(movie_ratings['budget'], labels = ['vlow', 'low', 'med', 'high', 'vhigh'], q = 5)

In [0]:
budget_ratings = movie_ratings[['title', 'budget', 'revenue', 'rating']].dropna()

In [0]:
budget_ratings.corr()

In [0]:
trace = go.Scatter(
    y = budget_ratings['rating'],
    x = budget_ratings['revenue'],
    mode = 'markers'
)

go.Figure(
    trace,
    layout_xaxis_title = "Budget",
    layout_yaxis_title = "Movie Rating",
    layout_title = "Movie Ratings by budget",
    
)

In [0]:
movie_ratings.shape

In [0]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA

In [0]:
all_ratings = movie_ratings[['vote_average','rating']].dropna()
all_ratings

In [0]:
all_ratings.isnull().sum()

In [0]:
scaler = StandardScaler()
x =scaler.fit_transform(all_ratings)

In [0]:
pca = PCA(1)

In [0]:
x = pca.fit_transform(x)

In [0]:
scaled_ratings = pd.DataFrame(x, index = all_ratings.index, columns=['scaled_rating'])

In [0]:
scaled_ratings

In [0]:
movie_ratings = movie_ratings.merge(scaled_ratings, left_index=True, right_index=True)

In [0]:
movie_ratings.corr()

In [0]:
enable_plotly_in_cell()
movie_ratings.groupby('year')['scaled_rating'].mean().iplot(kind='bar')

In [0]:
movie_ratings.groupby('year')['scaled_rating'].mean().iplot(kind='bar', title = "Scaled Rating by Year", xTitle="year", yTitle="Scaled Rating")

In [0]:
movie_ratings.groupby('decade')['scaled_rating'].mean().iplot(kind='bar')

In [0]:
budget_ratings = movie_ratings[['budget', 'rating']].dropna()

In [0]:
budget_ratings['q_budget'] = pd.qcut(budget_ratings['budget'], q = 5, labels = ['vlow', 'low', 'med', 'high', 'blockbuster'])


In [0]:
enable_plotly_in_cell()
budget_ratings.groupby('q_budget')['rating'].mean().iplot(kind='bar')

In [0]:
movie_ratings.nlargest(25, 'rating')

In [0]:
enable_plotly_in_cell()
trace = go.Scatter(
    x = movie_ratings["budget"],
    y = movie_ratings['revenue'],
    mode = "markers"
)

go.Figure(trace)

## Genre

In [0]:
genres = movies[['title','genres']]

In [0]:
import ast

In [27]:
genres['genres_ls'] = [
                    [d['name'] for d in ast.literal_eval(x) if d['name'] not in bad_genres ]
                    for x in genres['genres']
                    ]
genres['genres_ls']

id
862.0        [Animation, Comedy, Family]
8844.0      [Adventure, Fantasy, Family]
15602.0                [Romance, Comedy]
31357.0         [Comedy, Drama, Romance]
11862.0                         [Comedy]
                        ...             
439050.0                 [Drama, Family]
111109.0                         [Drama]
67758.0        [Action, Drama, Thriller]
227506.0                              []
461257.0                              []
Name: genres_ls, Length: 45463, dtype: object

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
mlb = MultiLabelBinarizer()

In [0]:
mlb_ = mlb.fit_transform(genres['genres_ls'])

In [0]:
bad_genres = [
           'Aniplex',
 'BROSTA TV',
 'Carousel Productions',   
  'GoHands',
   'Mardock Scramble Production Committee',
    'Odyssey Media',
     'Pulser Productions',
 'Rogue State',
  'Sentai Filmworks',
   'Telescene Film Group Productions',
 'The Cartel',
  'Vision View Entertainment',
]

#genre_set = genre_set.difference(bad_genres)

In [0]:
encoded_genres = pd.DataFrame(mlb_, columns = mlb.classes_, index = genres.index)

In [0]:
encoded_genres = clean_movies.merge(encoded_genres, **merge_keys)

In [43]:
for genre in mlb.classes_:
    print(genre, encoded_genres[encoded_genres[genre] == 1]['vote_average'].mean())

Action 5.75075674832266
Adventure 5.886064363743711
Animation 6.443680592279214
Comedy 5.979235709189001
Crime 6.107923892100158
Documentary 6.6706053550640005
Drama 6.187262592898546
Family 5.934633965068733
Fantasy 5.9245916114790065
Foreign 5.981445937300071
History 6.425392083644524
Horror 5.313143601998684
Music 6.355135135135147
Mystery 5.970215410107693
Romance 6.049242424242426
Science Fiction 5.472276231981214
TV Movie 5.67937853107345
Thriller 5.7423293172690695
War 6.298033044846587
Western 5.727872340425541


In [0]:
d = {g: [encoded_genres[encoded_genres[g] == 1]['vote_average'].mean(),
         encoded_genres[encoded_genres[g] == 1]['revenue'].sum()]
     for g in mlb.classes_}

In [0]:
df = pd.DataFrame.from_dict(d, orient='index',  columns = ['tMDb rating', 'total revenue'])

In [60]:
trace = go.Bar(
    x=pd.Series(d)
)

go.Figure(
    data = trace,
    layout_x)



In [76]:
alt.Chart(df).mark_bar().encode(
    x = df.index
)

IndexError: ignored